# Fine-Tuning QA model

https://www.youtube.com/watch?v=3XiJrn_8F9Q&t=1197s <br>
https://www.youtube.com/watch?v=6ppxd9lp-X0


dependency needed:
transformer
simpletransformer

In [17]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

""" # a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input) """

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [2]:
res

{'score': 0.21171477437019348,
 'start': 59,
 'end': 84,
 'answer': 'gives freedom to the user'}

In [3]:
#looking at RoBerta Question Answering
model

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

How to fine-tune a QA model
- we need GPU
- building a training script


In [6]:
import json
with open(r"data/QA_model/train.json", "r") as read_file:
    data = json.load(read_file)

#This dataset is used to train the model. The model learns by adjusting its weights based on the input-output pairs in this dataset.
#The training dataset is typically the largest subset. For supervised learning, this dataset consists of input data and the corresponding correct output.

In [7]:
data

[{'context': 'In this Order and Authorization, we grant, to the extent set forth below, the request of Kuiper Systems LLC (Kuiper or Amazon) to provide satellite services.\n            Operating 3,372 satellites in 102 orbital planes at altitudes of 590 km, 610 km, and 630 km in a circular orbit.\n            At 590 km, 30 orbital planes with 28 satellites per plane for a total of 840 satellites at inclination of 33 degree.\n            At 610 km, 42 orbital planes with 36 satellites per plane for a total of 1512 satellites at inclination of 42 degree.\n            At 630 km, 30 orbital planes with 34 satellites per plane for a total of 1020 satellite at inclination of 51.9 degree.\n            The constellation are require to launch and operate 50 percent of its satellites no later than July 30, 2026, and must launch the remaining space stations necessary to complete its authorized service constellation, place them in their assigned orbits, and operate each of them in accordance with 

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert the dataset to a more manageable format
def flatten_data(data):
    contexts = []
    questions = []
    answers = []
    for item in data:
        context = item['context']
        for qa in item['qas']:
            contexts.append(context)
            questions.append(qa['question'])
            answers.append(qa['answers'][0])  # since there's only one answer in your dataset
    return {"context": contexts, "question": questions, "answers": answers}

# Flatten your dataset
flattened_data = flatten_data(data)

c:\Users\zohar\.conda\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zohar\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
flattened_data

{'context': ['In this Order and Authorization, we grant, to the extent set forth below, the request of Kuiper Systems LLC (Kuiper or Amazon) to provide satellite services.\n            Operating 3,372 satellites in 102 orbital planes at altitudes of 590 km, 610 km, and 630 km in a circular orbit.\n            At 590 km, 30 orbital planes with 28 satellites per plane for a total of 840 satellites at inclination of 33 degree.\n            At 610 km, 42 orbital planes with 36 satellites per plane for a total of 1512 satellites at inclination of 42 degree.\n            At 630 km, 30 orbital planes with 34 satellites per plane for a total of 1020 satellite at inclination of 51.9 degree.\n            The constellation are require to launch and operate 50 percent of its satellites no later than July 30, 2026, and must launch the remaining space stations necessary to complete its authorized service constellation, place them in their assigned orbits, and operate each of them in accordance with 

In [11]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Modified preprocess function
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"]
        end_char = start_char + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

preprocessed_data = preprocess_function(flattened_data)

In [12]:
preprocessed_data

{'input_ids': [[101, 2054, 1005, 1055, 1996, 2171, 1997, 1996, 5871, 15300, 1996, 2194, 11014, 2000, 21296, 2030, 5452, 1029, 102, 1999, 2023, 2344, 1998, 20104, 1010, 2057, 3946, 1010, 2000, 1996, 6698, 2275, 5743, 2917, 1010, 1996, 5227, 1997, 13970, 15457, 2099, 3001, 11775, 1006, 13970, 15457, 2099, 2030, 9733, 1007, 2000, 3073, 5871, 2578, 1012, 4082, 1017, 1010, 4261, 2475, 14549, 1999, 9402, 13943, 9738, 2012, 21973, 1997, 25186, 2463, 1010, 19827, 2463, 1010, 1998, 23609, 2463, 1999, 1037, 8206, 8753, 1012, 2012, 25186, 2463, 1010, 2382, 13943, 9738, 2007, 2654, 14549, 2566, 4946, 2005, 1037, 2561, 1997, 28122, 14549, 2012, 21970, 1997, 3943, 3014, 1012, 2012, 19827, 2463, 1010, 4413, 13943, 9738, 2007, 4029, 14549, 2566, 4946, 2005, 1037, 2561, 1997, 16528, 2475, 14549, 2012, 21970, 1997, 4413, 3014, 1012, 2012, 23609, 2463, 1010, 2382, 13943, 9738, 2007, 4090, 14549, 2566, 4946, 2005, 1037, 2561, 1997, 9402, 2692, 5871, 2012, 21970, 1997, 4868, 1012, 1023, 3014, 1012, 1996, 1

In [14]:
from datasets import Dataset

# Convert flattened_data to a Dataset object
dataset = Dataset.from_dict(flattened_data)

# Process the dataset
tokenized_data = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)



Map:   0%|          | 0/42 [00:00<?, ? examples/s]

In [15]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Training

In [18]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
# Let's start training!
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=0.01,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=11,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

## Krish Naik

In [ ]:

import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:

### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

In [ ]:
### Remove output folder
!rm -rf outputs

In [ ]:
# Train the model
model.train_model(train, eval_data=test)

In [ ]:
# Evaluate the model
result, texts = model.eval_model(test)

In [ ]:
# Make predictions with the model
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]
    

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)